## **Preface**

This guide doesn't contain any python code, just raw `.usda` snippets.
I believe by the end of this guide, you will be able to build your own simple USD assets in raw `.usda`.

> The asset structures described in this guide are intended to be simple. They may not be ideal for production although they may work anyway.

## **Introduction**

The goal of using USD is to build consistent pipelines across production.

In my opinion, the first building block of any pipeline is how the assets are built/structured.

In this guide we are going to discuss a very simple asset structure.

## **Asset Structre Examples**

### **Example 1 : Geo and Mtl layers**
In this example: we have 3 layers. 
- `geo.usda` for our geometry
- `mtl.usda` for our materials
- `{asset-name}.usda` to wrap up `geo.usda` and `mtl.usda`

In [1]:
%%file examples/usd_asset_structure/example_1_geo_mtl/geo.usda
#usda 1.0

def Xform "Cube_Asset"
{
    def Scope "Geo"
    {
        def Cube "Cube"
        { }
    }
}

Overwriting examples/usd_asset_structure/example_1_geo_mtl/geo.usda


In [2]:
%%file examples/usd_asset_structure/example_1_geo_mtl/mtl.usda
#usda 1.0

def Xform "Cube_Asset"
{
    def Scope "Mtl"
    {
        def Material "Red"
        {
            token outputs:surface.connect = </Cube_Asset/Mtl/Red/Red_Preview.outputs:surface>

            def Shader "Red_Preview"
            {
                uniform token info:id = "UsdPreviewSurface"
                color3f inputs:diffuseColor = (1, 0, 0)
                token outputs:surface
            }
        }
    }

    over "Geo"
    {
        over "Cube" (
            prepend apiSchemas = ["MaterialBindingAPI"]
        )
        {
            rel material:binding = </Cube_Asset/Mtl/Red>
        }
    }
}

Overwriting examples/usd_asset_structure/example_1_geo_mtl/mtl.usda


In [3]:
%%file examples/usd_asset_structure/example_1_geo_mtl/cube_asset.usda
#usda 1.0

def Xform "Cube_Asset" (
    prepend references = [
        @./geo.usda@</Cube_Asset>,  # bring contents of Cube_Asset in geo.usda
        @./mtl.usda@</Cube_Asset>   # bring contents of Cube_Asset in mtl.usda
    ]
)
{}

Overwriting examples/usd_asset_structure/example_1_geo_mtl/cube_asset.usda


**Result:**

![](../resources/asset_structure/example_1_geo_mtl.png)


### **Example 2 : Geo and Mtl layers as payload**
In this example: we have 4 layers. 
- `geo.usda` for our geometry
- `mtl.usda` for our materials
- `payload.usda` to wrap up `geo.usda` and `mtl.usda`
- `{asset-name}.usda` to payload `payload.usda`

In [4]:
%%file examples/usd_asset_structure/example_2_geo_mtl_payload/geo.usda
#usda 1.0

def Xform "Cube_Asset"
{
    def Scope "Geo"
    {
        def Cube "Cube"
        { }
    }
}

Overwriting examples/usd_asset_structure/example_2_geo_mtl_payload/geo.usda


In [5]:
%%file examples/usd_asset_structure/example_2_geo_mtl_payload/mtl.usda
#usda 1.0

def Xform "Cube_Asset"
{
    def Scope "Mtl"
    {
        def Material "Red"
        {
            token outputs:surface.connect = </Cube_Asset/Mtl/Red/Red_Preview.outputs:surface>

            def Shader "Red_Preview"
            {
                uniform token info:id = "UsdPreviewSurface"
                color3f inputs:diffuseColor = (1, 0, 0)
                token outputs:surface
            }
        }
    }

    over "Geo"
    {
        over "Cube" (
            prepend apiSchemas = ["MaterialBindingAPI"]
        )
        {
            rel material:binding = </Cube_Asset/Mtl/Red>
        }
    }
}

Overwriting examples/usd_asset_structure/example_2_geo_mtl_payload/mtl.usda


In [6]:
%%file examples/usd_asset_structure/example_2_geo_mtl_payload/payload.usda
#usda 1.0

def Xform "Cube_Asset" (
    prepend references = [
        @./geo.usda@</Cube_Asset>,  # bring contents of Cube_Asset in geo.usda
        @./mtl.usda@</Cube_Asset>   # bring contents of Cube_Asset in mtl.usda
    ]
)
{}

Overwriting examples/usd_asset_structure/example_2_geo_mtl_payload/payload.usda


In [7]:
%%file examples/usd_asset_structure/example_2_geo_mtl_payload/cube_asset.usda
#usda 1.0

def Xform "Cube_Asset" (
    prepend payload = @./payload.usda@</Cube_Asset>
)
{}

Overwriting examples/usd_asset_structure/example_2_geo_mtl_payload/cube_asset.usda


**Result:**

![](../resources/asset_structure/example_2_geo_mtl_payload.gif)


### **Example 3 : Geo with variants and Mtl layers as payload**
In this example: we have 4 layers. 
- `geo.usda` for our geometry variants
- `mtl.usda` for our materials
- `payload.usda` to wrap up `geo.usda` and `mtl.usda`
- `{asset-name}.usda` to payload `payload.usda` with lofted variants.

> `Lofing` a property means to make it available outside the payload.
>
> Lofted variant set: make variant options available even when unloading the payload.

In [8]:
%%file examples/usd_asset_structure/example_3_variant_geo_mtl_payload/geo.usda
#usda 1.0

def Xform "Cube_Asset" (
    variants = {
        string model = "normal"
    }
    prepend variantSets = "model"
)
{
    def Scope "Geo"
    {
        def Cube "Cube"
        {
            float3 xformOp:scale
            uniform token[] xformOpOrder = ["xformOp:scale"]
        }
    }
    variantSet "model" = {
        "normal" {
            over "Geo"
            {
                over "Cube"
                {
                    float3 xformOp:scale = (1, 1, 1)
                }
            }

        }
        "short" {
            over "Geo"
            {
                over "Cube"
                {
                    float3 xformOp:scale = (1, 0.5, 1)
                }
            }

        }
        "tall" {
            over "Geo"
            {
                over "Cube"
                {
                    float3 xformOp:scale = (1, 2, 1)
                }
            }

        }
    }
}


Overwriting examples/usd_asset_structure/example_3_variant_geo_mtl_payload/geo.usda


In [9]:
%%file examples/usd_asset_structure/example_3_variant_geo_mtl_payload/mtl.usda
#usda 1.0

def Xform "Cube_Asset"
{
    def Scope "Mtl"
    {
        def Material "Red"
        {
            token outputs:surface.connect = </Cube_Asset/Mtl/Red/Red_Preview.outputs:surface>

            def Shader "Red_Preview"
            {
                uniform token info:id = "UsdPreviewSurface"
                color3f inputs:diffuseColor = (1, 0, 0)
                token outputs:surface
            }
        }
    }

    over "Geo"
    {
        over "Cube" (
            prepend apiSchemas = ["MaterialBindingAPI"]
        )
        {
            rel material:binding = </Cube_Asset/Mtl/Red>
        }
    }
}

Overwriting examples/usd_asset_structure/example_3_variant_geo_mtl_payload/mtl.usda


In [10]:
%%file examples/usd_asset_structure/example_3_variant_geo_mtl_payload/payload.usda
#usda 1.0

def Xform "Cube_Asset" (
    prepend references = [
        @./geo.usda@</Cube_Asset>,  # bring contents of Cube_Asset in geo.usda
        @./mtl.usda@</Cube_Asset>   # bring contents of Cube_Asset in mtl.usda
    ]
)
{}

Overwriting examples/usd_asset_structure/example_3_variant_geo_mtl_payload/payload.usda


In [11]:
%%file examples/usd_asset_structure/example_3_variant_geo_mtl_payload/cube_asset.usda
#usda 1.0

def Xform "Cube_Asset" (
    prepend payload = @./payload.usda@</Cube_Asset>

    # Loft Variant Set
    variants = {
        string model = "normal"
    }
    prepend variantSets = "model"
)
{
    # Loft Variant options
    variantSet "model" = {
        "normal" {

        }
        "short" {

        }
        "tall" {

        }
    }
}

Overwriting examples/usd_asset_structure/example_3_variant_geo_mtl_payload/cube_asset.usda


**Result:**

![](../resources/asset_structure/example_3_variant_geo_mtl_payload.gif)

### **Example 4 : variant Geo and variant Mtl layers as payload**
In this example: we have 4 layers. 
- `geo.usda` for our geometry variants
- `mtl.usda` for our materials variants
- `payload.usda` to wrap up `geo.usda` and `mtl.usda`
- `{asset-name}.usda` to payload `payload.usda` with lofted variants.

> This structure assumes that all geo variants have the same prims hierarchy and names.

![](../resources/asset_structure/asset_structure.png)

In [12]:
%%file examples/usd_asset_structure/example_4_variant_geo_variant_mtl_payload/geo.usda
#usda 1.0

def Xform "Cube_Asset" (
    variants = {
        string model = "normal"
    }
    prepend variantSets = "model"
)
{
    def Scope "Geo"
    {
        def Cube "Cube"
        {
            float3 xformOp:scale
            uniform token[] xformOpOrder = ["xformOp:scale"]
        }
    }
    variantSet "model" = {
        "normal" {
            over "Geo"
            {
                over "Cube"
                {
                    float3 xformOp:scale = (1, 1, 1)
                }
            }

        }
        "short" {
            over "Geo"
            {
                over "Cube"
                {
                    float3 xformOp:scale = (1, 0.5, 1)
                }
            }

        }
        "tall" {
            over "Geo"
            {
                over "Cube"
                {
                    float3 xformOp:scale = (1, 2, 1)
                }
            }

        }
    }
}

Overwriting examples/usd_asset_structure/example_4_variant_geo_variant_mtl_payload/geo.usda


In [13]:
%%file examples/usd_asset_structure/example_4_variant_geo_variant_mtl_payload/mtl.usda
#usda 1.0

def Xform "Cube_Asset" (
    variants = {
        string material = "blue"
    }
    prepend variantSets = "material"
)
{
    def Scope "Mtl"
    {
        def Material "Red"
        {
            token outputs:surface.connect = </Cube_Asset/Mtl/Red/Red_Preview.outputs:surface>

            def Shader "Red_Preview"
            {
                uniform token info:id = "UsdPreviewSurface"
                color3f inputs:diffuseColor = (1, 0, 0)
                token outputs:surface
            }
        }

        def Material "Green"
        {
            token outputs:surface.connect = </Cube_Asset/Mtl/Green/Green_Preview.outputs:surface>

            def Shader "Green_Preview"
            {
                uniform token info:id = "UsdPreviewSurface"
                color3f inputs:diffuseColor = (0, 1, 0)
                token outputs:surface
            }
        }

        def Material "Blue"
        {
            token outputs:surface.connect = </Cube_Asset/Mtl/Blue/Blue_Preview.outputs:surface>

            def Shader "Blue_Preview"
            {
                uniform token info:id = "UsdPreviewSurface"
                color3f inputs:diffuseColor = (0, 0, 1)
                token outputs:surface
            }
        }
    }

    variantSet "material" = {
        "blue" {
            over "Geo"
            {
                over "Cube" (
                    prepend apiSchemas = ["MaterialBindingAPI"]
                )
                {
                    rel material:binding = </Cube_Asset/Mtl/Blue>
                }
            }

        }
        "green" {
            over "Geo"
            {
                over "Cube" (
                    prepend apiSchemas = ["MaterialBindingAPI"]
                )
                {
                    rel material:binding = </Cube_Asset/Mtl/Green>
                }
            }

        }
        "red" {
            over "Geo"
            {
                over "Cube" (
                    prepend apiSchemas = ["MaterialBindingAPI"]
                )
                {
                    rel material:binding = </Cube_Asset/Mtl/Red>
                }
            }

        }
    }
}



Overwriting examples/usd_asset_structure/example_4_variant_geo_variant_mtl_payload/mtl.usda


In [14]:
%%file examples/usd_asset_structure/example_4_variant_geo_variant_mtl_payload/payload.usda
#usda 1.0

def Xform "Cube_Asset" (
    prepend references = [
        @./geo.usda@</Cube_Asset>,  # bring contents of Cube_Asset in geo.usda
        @./mtl.usda@</Cube_Asset>   # bring contents of Cube_Asset in mtl.usda
    ]
)
{}

Overwriting examples/usd_asset_structure/example_4_variant_geo_variant_mtl_payload/payload.usda


In [15]:
%%file examples/usd_asset_structure/example_4_variant_geo_variant_mtl_payload/cube_asset.usda
#usda 1.0

def Xform "Cube_Asset" (
    prepend payload = @./payload.usda@</Cube_Asset>

    # Loft Variant Sets
    variants = {
        string material = "red"
        string model = "normal"
    }
    prepend variantSets = ["model", "material"]
)
{
    # Loft Variant options
    variantSet "model" = {
        "normal" {

        }
        "short" {

        }
        "tall" {

        }
    }

    variantSet "material" = {
        "blue" {

        }
        "green" {

        }
        "red" {

        }
    }
}

Overwriting examples/usd_asset_structure/example_4_variant_geo_variant_mtl_payload/cube_asset.usda


**Result:**

![](../resources/asset_structure/example_4_variant_geo_variant_mtl_payload.gif)

## USD Asset Structure guides

For Further reading, dive into these guides.

- [Principles of Scalable Asset Structure | Nvidia](https://docs.omniverse.nvidia.com/usd/latest/learn-openusd/independent/asset-structure-principles.html#principles-of-scalable-asset-structure-in-openusd)
- [Guidelines for Structuring USD Assets | Chris Rydalch](https://wiki.aswf.io/display/WGUSD/Guidelines+for+Structuring+USD+Assets)
- [Component Builder | SideFx](https://www.sidefx.com/docs/houdini/solaris/component_builder.html)
- [ALab File Structure | ALA](https://usd-alab2.s3.amazonaws.com/documentation.html)